In [ ]:
import numpy as np
from scipy.signal import hilbert
import time

# Define num_signals before using it as a default argument
num_signals = 8

# Define the features dictionary with the required values
features = {
    "min_volt": 1e-6,  # 1 microvolt
    "max_volt": 8e-6,  # 8 microvolts
    "variability_factor": 0.1,  # Direct mapping of player movement to variability factor, normalized to a 0-1 scale
    "variance": 0.01,  # Mapping door state to variance feature
    "std_dev": 0.05,  # Mapping enemy type to standard deviation feature
    "rms_value": 0.02,  # Player health state affects the RMS value feature
    "num_peaks": 5,  # Number of peaks determined by exploring states
    "peak_height": 0.05,  # Peak height influenced by level state
    "fractal_dimension": 1.5,  # Action states influence the fractal dimension
    "window_size": 10,  # Window size feature influenced by wall states
    "target_rate": 0.1,  # Target rate is determined by the presence of any enemy type
    "min_freq": 0.5,  # Minimum frequency affected by player movement, demonstrating a range for frequency based on movement
    "max_freq": 1.5,  # Maximum frequency influenced by player health state, indicating a dynamic range based on health
    "blend_factor": 0.2,  # Static blend factor as a static state
    "global_sync_level": 0.1,  # Global sync level determined by action state, reflecting synchronization needs
    "pairwise_sync_level": 0.2,  # Pairwise sync level affected by door state, indicating sync adjustments based on environmental factors
    "sync_factor": 0.05,  # Sync factor as a static value for simplicity
    "influence_factor": 0.1,  # Influence factor derived from enemy type, representing external influence levels
    "max_influence": 0.02,  # Maximum influence as a static maximum for the presence of any enemy
    "centroid_factor": 0.1,  # Centroid factor and edge density factor as placeholders for sensory data encoding
    "edge_density_factor": 0.2,  # Centroid factor and edge density factor as placeholders for sensory data encoding
    "complexity_factor": 0.1,  # Example value for complexity factor in FFT
    "evolution_rate": 0.05,  # Evolution rate as a static value for dynamic environmental changes
    "low_freq": 0.5,  # Low frequency ranges influenced by exploring states
    "high_freq": 2.0,  # High frequency ranges influenced by level states
    "causality_strength": 0.1,  # Causality strength as a static value for interaction effects
    "num_imfs": 5,  # Number of intrinsic mode functions (IMFs) as a static value for interaction effects
}

# Function to generate base ECoG-like signals within the specified range
def generate_ecog_like_base_signals(length, num_signals=num_signals):
    signals = np.random.uniform(min_volt, max_volt, (num_signals, length))
    return signals

# Function to scale signals to a specific bit depth
def scale_signals_to_bit_depth(modified_signals, bit_depth):
    # Ensure modified_signals is a floating-point array to allow for intermediate float operations
    modified_signals = modified_signals.astype(np.float64)
    
    # Normalize signals to range [0, 1]
    min_signal = np.min(modified_signals)
    max_signal = np.max(modified_signals)
    normalized_signals = (modified_signals - min_signal) / (max_signal - min_signal)
    
    # Scale to the new range determined by bit depth
    max_value = (2 ** bit_depth) - 1
    modified_signals = normalized_signals * max_value

    return modified_signals

# Function to apply amplitude variability while keeping the signals within the specified range
def apply_amplitude_variability(modified_signals, variability_factor):
    # Apply amplitude variability to each signal individually
    for i in range(modified_signals.shape[0]):
        # Calculate the scaling factor for each signal
        current_std = np.std(modified_signals[i])
        if current_std == 0:
            continue  # Skip if standard deviation is zero to avoid division by zero
        scaling_factor = np.sqrt(variability_factor) / current_std

        # Generate scaled noise for each signal
        scaled_noise = np.random.normal(0, scaling_factor, modified_signals.shape[1])

        # Add the scaled noise to the signal
        modified_signals[i] += scaled_noise

    # Scale the entire signal set to fit within the min_volt and max_volt range
    max_signal = np.max(modified_signals)
    min_signal = np.min(modified_signals)
    modified_signals = ((modified_signals - min_signal) / (max_signal - min_signal)) * (max_volt - min_volt) + min_volt

    return modified_signals

# Function to apply variance while keeping the signals within the specified range
def apply_variance(modified_signals, variance):
    for i in range(modified_signals.shape[0]):
        # Generate noise for each signal based on the specified variance
        noise = np.random.normal(0, np.sqrt(variance), modified_signals.shape[1])

        # Add the noise to the signal
        modified_signals[i] += noise

    # Scale the entire signal set to fit within the min_volt and max_volt range
    max_signal = np.max(modified_signals)
    min_signal = np.min(modified_signals)
    modified_signals = ((modified_signals - min_signal) / (max_signal - min_signal)) * (max_volt - min_volt) + min_volt

    return modified_signals

# Function to apply signal with standard deviation while keeping the signals within the specified range
def apply_signal_with_std(modified_signals, std_dev):
    for i in range(modified_signals.shape[0]):
        # Calculate the scaling factor for each individual signal
        current_std = np.std(modified_signals[i])
        if current_std == 0:
            continue  # Skip if standard deviation is zero to avoid division by zero
        scaling_factor = std_dev / current_std

        # Apply the scaled noise to each individual signal
        modified_signals[i] += np.random.normal(0, scaling_factor * std_dev, modified_signals.shape[1])

    # Scale the entire signal set to fit within the min_volt and max_volt range
    max_signal = np.max(modified_signals)
    min_signal = np.min(modified_signals)
    modified_signals = ((modified_signals - min_signal) / (max_signal - min_signal)) * (max_volt - min_volt) + min_volt

    return modified_signals

# Function to apply signal with RMS value
def apply_signal_with_rms(modified_signals, rms_value):
    current_rms = np.sqrt(np.mean(modified_signals**2, axis=1, keepdims=True))
    scaling_factor = np.minimum(1.0, rms_value / current_rms)
    modified_signals *= scaling_factor
    return modified_signals

# Function to add peaks
def add_peaks(modified_signals, num_peaks, peak_height):
    if len(modified_signals) == 0:
        raise ValueError("Input modified_signals array must not be empty")
    num_signals, signal_length = modified_signals.shape
    scaling_factor = np.maximum(min_volt / np.min(modified_signals), max_volt / np.max(modified_signals))
    
    for i in range(num_signals):
        for _ in range(num_peaks):
            peak_position = np.random.randint(0, signal_length)
            modified_signals[i, peak_position] += peak_height * scaling_factor
            # Ensure that the modified signal does not exceed the specified range
            modified_signals[i, peak_position] = np.clip(modified_signals[i, peak_position], min_volt, max_volt)
            
    return modified_signals

# Function to apply moving average
def apply_moving_average(modified_signals, window_size):  
    for i in range(len(modified_signals)):
        length = len(modified_signals[i])
        if length >= window_size:
            cumsum_vec = np.cumsum(np.insert(modified_signals[i], 0, 0))
            moving_average = (cumsum_vec[window_size:] - cumsum_vec[:-window_size]) / window_size
            # Apply the moving average only to the middle part of the signal
            modified_signals[i, window_size//2:-window_size//2+1] = moving_average
    return modified_signals

# Function to apply fractal structure
def apply_fractal_structure(modified_signals, fractal_dimension):
    length = modified_signals.shape[1]
    fBm_signal = np.zeros_like(modified_signals)

    # Calculate Hurst exponent from fractal dimension
    hurst_exponent = 2 - fractal_dimension

    for i in range(1, length):
        scale = (i ** (2 * hurst_exponent)) - ((i-1) ** (2 * hurst_exponent))
        fBm_signal[:, i] = fBm_signal[:, i-1] + np.random.normal(0, np.sqrt(scale), modified_signals.shape[0])

    # Scale fBm_signal to match the original signal's range
    max_signal = np.max(fBm_signal, axis=1, keepdims=True)
    min_signal = np.min(fBm_signal, axis=1, keepdims=True)
    fBm_signal = (fBm_signal - min_signal) / (max_signal - min_signal)
    fBm_signal *= (max_volt - min_volt)
    fBm_signal += min_volt

    return modified_signals
    
# Function to apply zero-crossing rate
def apply_zero_crossing_rate(modified_signals, target_rate):
    def calculate_zero_crossing_rate(sig):
        zero_crossings = np.where(np.diff(np.sign(sig)))[0]
        return len(zero_crossings) / len(sig)

    for i in range(len(modified_signals)):
        current_rate = calculate_zero_crossing_rate(modified_signals[i])
        factor = target_rate / current_rate if current_rate > 0 else 1.0
        modified_signals[i] *= factor
    return modified_signals

# Function to apply Arnold tongues
def apply_arnold_tongues(modified_signals, min_freq, max_freq, blend_factor):
    num_signals, signal_length = modified_signals.shape

    for i in range(num_signals):
        # Generate a random K within a more controlled range
        K = np.random.uniform(0.1 * max_volt, 0.5 * max_volt)  # Reduced amplitude range

        # Generate an Arnold tongue pattern with the specified frequency range
        omega = np.random.uniform(min_freq, max_freq)  # Use the provided frequency range
        t = np.linspace(0, 2 * np.pi, signal_length)
        arnold_tongue_pattern = np.sin(t * omega) * K
        
        # Introduce additional harmonics or noise to the pattern
        harmonics = np.random.choice([2, 3, 4], size=1)
        noise = np.random.uniform(-0.1 * K, 0.1 * K, size=signal_length)
        arnold_tongue_pattern += (np.sin(t * omega * harmonics) + noise) * K * 0.5

        # Blend the Arnold tongue pattern with the original signal
        modified_signals[i, :] = (1 - blend_factor) * modified_signals[i, :] + blend_factor * arnold_tongue_pattern

    # Normalize the signals
    max_signal = np.max(modified_signals)
    min_signal = np.min(modified_signals)
    normalized_signals = ((modified_signals - min_signal) / (max_signal - min_signal)) * (max_volt - min_volt) + min_volt

    return normalized_signals

def apply_phase_synchronization(modified_signals, global_sync_level, pairwise_sync_level, sync_factor=0.05):
    num_signals, length = modified_signals.shape
    common_phase = np.linspace(0, 2 * np.pi * global_sync_level, length)

    # Apply global synchronization
    for i in range(num_signals):
        signal_fft = np.fft.fft(modified_signals[i])
        amplitude = np.abs(signal_fft)
        phase = np.angle(signal_fft)
        
        # Adjust phase globally
        global_phase_shift = sync_factor * np.interp(common_phase, (common_phase.min(), common_phase.max()), (-np.pi, np.pi))
        adjusted_phase = phase + global_phase_shift
        modified_signals[i] = np.fft.ifft(amplitude * np.exp(1j * adjusted_phase)).real

    # Apply pairwise synchronization more selectively
    for i in range(num_signals):
        for j in range(i + 1, num_signals):
            # Calculate pairwise phase difference
            phase_diff = np.angle(np.fft.fft(modified_signals[i])) - np.angle(np.fft.fft(modified_signals[j]))
            phase_diff_adjustment = np.interp(phase_diff, (-np.pi, np.pi), (-pairwise_sync_level, pairwise_sync_level))

            # Apply the phase adjustment selectively
            signal_fft_i = np.fft.fft(modified_signals[i])
            signal_fft_j = np.fft.fft(modified_signals[j])
            adjusted_phase_i = np.angle(signal_fft_i) + phase_diff_adjustment * sync_factor
            adjusted_phase_j = np.angle(signal_fft_j) - phase_diff_adjustment * sync_factor

            # Apply the adjusted phases
            modified_signals[i] = np.fft.ifft(np.abs(signal_fft_i) * np.exp(1j * adjusted_phase_i)).real
            modified_signals[j] = np.fft.ifft(np.abs(signal_fft_j) * np.exp(1j * adjusted_phase_j)).real

    # Rescale the signals
    max_signal = np.max(modified_signals)
    min_signal = np.min(modified_signals)
    modified_signals = ((modified_signals - min_signal) / (max_signal - min_signal)) * (max_volt - min_volt) + min_volt

    return modified_signals

def apply_transfer_entropy(modified_signals, influence_factor, max_influence):
    num_signals = len(modified_signals)
    max_length = max(len(signal) for signal in modified_signals)
    
    # Normalize interaction weights to sum up to the influence factor
    interaction_weights = np.random.uniform(0, 1, (num_signals, num_signals))
    interaction_weights /= interaction_weights.sum(axis=1, keepdims=True)
    interaction_weights *= influence_factor
    
    for i in range(1, max_length):
        # Calculate the influenced signals
        influenced_signals = np.dot(interaction_weights, [signal[i - 1] if i < len(signal) else signal[-1] for signal in modified_signals])
        
        # Calculate the maximum possible influence to stay within the specified voltage range
        max_possible_influence = (max_volt - min_volt) / max_length
        
        # Scale down the influenced signals if necessary
        if np.max(np.abs(influenced_signals)) > max_possible_influence:
            scaling_factor = max_possible_influence / np.max(np.abs(influenced_signals))
            influenced_signals *= scaling_factor
        
        # Add the influenced signals to the original signals
        for j in range(num_signals):
            if i < len(modified_signals[j]):
                modified_signals[j][i] += influenced_signals[j]

    return modified_signals

# Function to apply Hilbert-Huang transform
def apply_hilbert_huang(modified_signals):
    for i in range(len(modified_signals)):
        signal = modified_signals[i]
        
        # Calculate the amplitude for the low-frequency component (lowest 10%)
        low_freq_amplitude = 0.05 * (max_volt - min_volt)
        low_freq_signal = low_freq_amplitude * np.sin(2 * np.pi * 0.1 * np.arange(len(signal)))
        
        # Calculate the amplitude for the high-frequency component (highest 10%)
        high_freq_amplitude = 0.05 * (max_volt - min_volt)
        high_freq_noise = high_freq_amplitude * np.random.randn(len(signal))
        
        # Combine the components
        modulated_signal = signal + low_freq_signal + high_freq_noise

        # Clip the signal to the specified range
        modulated_signal = np.clip(modulated_signal, min_volt, max_volt)

        modified_signals[i] = modulated_signal

    return modified_signals

# Function to apply spectral centroids
def apply_spectral_centroids(modified_signals, centroid_factor, edge_density_factor):
    for i in range(modified_signals.shape[0]):
        fft_spectrum = np.fft.fft(modified_signals[i])
        freq = np.fft.fftfreq(len(modified_signals[i]))
        centroid = np.sum(freq * np.abs(fft_spectrum)) / np.sum(np.abs(fft_spectrum))
        edge_density = np.sum(np.abs(np.diff(fft_spectrum))) / np.sum(np.abs(fft_spectrum))
        adjusted_spectrum = fft_spectrum * (centroid_factor * centroid + edge_density_factor * edge_density)
        modified_signals[i] = np.fft.ifft(adjusted_spectrum).real
    return modified_signals
    
# Function to apply dynamic time warping
def apply_dynamic_time_warping(modified_signals, reference_signal, warping_factor, min_volt, max_volt):
    modified_signals_aligned = []

    reference_signal = np.array(reference_signal)  # Convert the reference signal to a numpy array

    for signal in modified_signals:
        signal = np.array(signal)  # Convert the signal to a numpy array

        # Perform dynamic time warping
        signal_length = len(signal)
        time_vector = np.linspace(0, 1, signal_length)
        reference_time_vector = np.linspace(0, 1, len(reference_signal)) * warping_factor
        modified_signal = np.interp(time_vector, reference_time_vector, reference_signal)

        # Normalize the signal to stay within the desired range
        current_min = np.min(modified_signal)
        current_max = np.max(modified_signal)

        # Calculate the scaling factor to keep the signal within the specified range
        scaling_factor = (max_volt - min_volt) / (current_max - current_min)

        # Apply the scaling factor while preserving the signal's shape
        modified_signal = (modified_signal - current_min) * scaling_factor + min_volt

        modified_signals_aligned.append(modified_signal)

    return np.array(modified_signals)
    
def apply_fft(modified_signals, complexity_factor):
    modified_signals_complexity = []

    for signal in modified_signals:
        # Apply FFT to each signal
        spectrum = np.fft.fft(signal)
        freq = np.fft.fftfreq(len(signal))

        # Calculate the scaling factor to keep the signal within the range
        current_max = np.max(signal)
        current_min = np.min(signal)
        scaling_factor = max_volt / (current_max - current_min)

        # Apply complexity modification
        modified_spectrum = spectrum * (1 + np.random.randn(len(spectrum)) * complexity_factor)
        
        # Transform back to time domain
        modified_signal = np.fft.ifft(modified_spectrum).real
        
        # Scale the signal while preserving the range
        modified_signal = (modified_signal - current_min) * scaling_factor + min_volt
        modified_signals_complexity.append(modified_signal)

    return np.array(modified_signals)

def apply_signal_evolution(modified_signals, evolution_rate):
    modified_signals_evolution = np.zeros_like(modified_signals)
    for i in range(modified_signals.shape[0]):
        modified_signals_evolution[i, 0] = np.random.randn()
        for t in range(1, modified_signals.shape[1]):
            modified_signals_evolution[i, t] = modified_signals_evolution[i, t - 1] + evolution_rate * np.random.randn()
    return modified_signals

def apply_phase_amplitude_coupling(modified_signals, low_freq, high_freq):
    modified_signals_pac = np.zeros_like(modified_signals)
    time = np.linspace(0, 1, modified_signals.shape[1])
    for i in range(modified_signals.shape[0]):
        low_freq_signal = np.sin(2 * np.pi * low_freq * time)
        high_freq_signal = np.sin(2 * np.pi * high_freq * time)
        modified_signals_pac[i] = (1 + low_freq_signal) * high_freq_signal
    return modified_signals

def apply_granger_causality(modified_signals, causality_strength):
    modified_signals_granger = np.copy(modified_signals)
    num_signals, signal_length = modified_signals.shape
    for i in range(1, num_signals):
        for j in range(num_signals):
            if i != j:
                causal_effect = modified_signals[j, :-i] * causality_strength
                modified_signals_granger[i, i:] += causal_effect
    return modified_signals

def apply_multivariate_empirical_mode_decomposition(modified_signals, num_imfs):
    num_channels, signal_length = modified_signals.shape
    memd_signals = np.zeros((num_channels, num_imfs, signal_length))
    for channel in range(num_channels):
        for imf_idx in range(num_imfs):
            imf = np.sin(2 * np.pi * (imf_idx + 1) * np.linspace(0, 1, signal_length))
            amplitude_scaling = np.random.uniform(0.5, 2.0)
            memd_signals[channel, imf_idx, :] = amplitude_scaling * imf
    modified_signals_memd = modified_signals + np.sum(memd_signals, axis=1)
    return modified_signals

def apply_normalized_states(modified_signals, modification_factor, matrix_size, value_range, num_matrices, eigenvalue_subset):
    num_signals, signal_length = modified_signals.shape
    modified_signals = modified_signals.copy()

    # Function to generate a parameterized random Hermitian matrix
    def generate_parameterized_hermitian_matrix(size, value_range):
        A = np.random.uniform(value_range[0], value_range[1], (size, size)) + \
            1j * np.random.uniform(value_range[0], value_range[1], (size, size))
        return A + A.conj().T

    density_diagonals = []
    for _ in range(num_matrices):
        hermitian_matrix = generate_parameterized_hermitian_matrix(matrix_size, value_range)
        eigenvalues, eigenvectors = np.linalg.eigh(hermitian_matrix)
        eigenvalues = eigenvalues[eigenvalue_subset]
        eigenvectors = eigenvectors[:, eigenvalue_subset]
        density_matrix = np.dot(eigenvectors, np.dot(np.diag(eigenvalues), eigenvectors.T.conj()))
        density_diagonal = np.diag(density_matrix).real
        density_diagonals.append(density_diagonal)

    # Apply modifications using the density diagonals
    for i, signal in enumerate(modified_signals):
        density_diagonal = density_diagonals[i % len(density_diagonals)]
        signal_modification = np.interp(signal, (np.min(signal), np.max(signal)), (np.min(density_diagonal), np.max(density_diagonal)))
        modified_signals[i] = (1 - modification_factor) * signal + modification_factor * signal_modification

    # Normalize the signals
    max_signal = np.max(modified_signals)
    min_signal = np.min(modified_signals)
    modified_signals = ((modified_signals - min_signal) / (max_signal - min_signal)) * (max_volt - min_volt) + min_volt

    return modified_signals

def final_modified_scale_signals_to_bit_depth(modified_signals, bit_depth):
    # Assuming that the desired final range is defined by global variables min_volt and max_volt
    min_val, max_val = min_volt, max_volt

    # Calculate the current range of the signals
    max_signal = np.max(modified_signals)
    min_signal = np.min(modified_signals)

    # Scale the signals to the specified bit depth
    # The formula maps the current range to the desired range [min_val, max_val]
    scaled_signals = ((modified_signals - min_signal) / (max_signal - min_signal)) * (max_val - min_val) + min_val

    return scaled_signals

min_volt = features["min_volt"]
max_volt = features["max_volt"]

# Define the list of transformation functions using features
transformations = [
    lambda x: scale_signals_to_bit_depth(x, bit_depth),
    lambda x: apply_amplitude_variability(x, features["variability_factor"]),
    lambda x: apply_variance(x, features["variance"]),
    lambda x: apply_signal_with_std(x, features["std_dev"]),
    lambda x: apply_signal_with_rms(x, features["rms_value"]),
    lambda x: add_peaks(x, features["num_peaks"], features["peak_height"]),
    lambda x: apply_fractal_structure(x, features["fractal_dimension"]),
    lambda x: apply_moving_average(x, features["window_size"]),
    lambda x: apply_zero_crossing_rate(x, features["target_rate"]),
    lambda x: apply_arnold_tongues(x, features["min_freq"], features["max_freq"], features["blend_factor"]),
    lambda x: apply_phase_synchronization(x, features["global_sync_level"], features["pairwise_sync_level"], features["sync_factor"]),
    lambda x: apply_transfer_entropy(x, features["influence_factor"], features["max_influence"]),
    lambda x: apply_hilbert_huang(x),
    lambda x: apply_spectral_centroids(x, features["centroid_factor"], features["edge_density_factor"]),
    lambda x: apply_dynamic_time_warping(x, reference_signal=np.mean(x, axis=0), warping_factor=0.5, min_volt=min_volt, max_volt=max_volt),
    lambda x: apply_fft(x, features["complexity_factor"]),
    lambda x: apply_signal_evolution(x, features["evolution_rate"]),
    lambda x: apply_phase_amplitude_coupling(x, features["low_freq"], features["high_freq"]),
    lambda x: apply_granger_causality(x, features["causality_strength"]),
    lambda x: apply_multivariate_empirical_mode_decomposition(x, features["num_imfs"]),
    lambda x: apply_normalized_states(x, modification_factor=0.1, matrix_size=10, value_range=(-1, 1), num_matrices=5, eigenvalue_subset=slice(0, 5)),
]

# Main function to generate and transform signals
def generate_transformed_signals(signal_length, num_signals, transformation_functions):
    modified_signals = generate_ecog_like_base_signals(signal_length, num_signals)

    for transform in transformation_functions:
        modified_signals = transform(modified_signals)
        # Convert list to NumPy array if necessary
        if isinstance(modified_signals, list):
            modified_signals = np.array(modified_signals)

        if np.isnan(modified_signals).any() or np.isinf(modified_signals).any():
            print(f"NaNs or Infinities found after {transform.__name__}")
        print(f"Range after {transform.__name__}: {modified_signals.min()} to {modified_signals.max()}")

    # Normalize and scale to full 16-bit range
    modified_signals = final_modified_scale_signals_to_bit_depth(modified_signals, 16)
    print(f"Scaled range: {modified_signals.min()} to {modified_signals.max()}")

    return modified_signals

# Main execution
if __name__ == "__main__":
    bit_depth = 16        # Bit depth of the signals
    num_signals = 8      # Number of signals
    fs = 500              # Sampling rate in Hz
    duration = 3600          # Duration in seconds
    length = fs * duration

    # Global variables for amplitude range
    min_volt = 1
    max_volt = 8  

    # Calculate the length of the signals
    length = fs * duration

    # Generate and transform the ECoG-like signals
    transformed_signals = generate_transformed_signals(length, num_signals, transformations)

    # Convert transformed_signals to a NumPy array to access the shape attribute
    transformed_signals_array = np.array(transformed_signals)

    # Check the shape and a sample of the generated signals
    print(f"Shape of generated ECoG-like signals: {transformed_signals_array.shape}")
    for i in range(8):
        print(f"Sample of signal {i + 1}: {transformed_signals_array[i][:50]}")

    # Check the shape and a sample of the generated signals
    print(f"Shape of generated ECoG-like signals: {transformed_signals_array.shape}")
    print(f"Sample of first signal: {transformed_signals_array[0][:100]}")

In [ ]:
import numpy as np
import time
from neuroplatform import StimParam, IntanSoftware, Trigger, Experiment

# Set the experiment token and initialize the experiment
token = "XSALK6J9C4"
exp = Experiment(token)
print(f'Electrodes: {32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63}')  # Electrodes that you can use

num_read_elecs = 32
num_stim_elecs = 8  # Number of neural stimulators

# Define the stimulation pattern with manual chunks
stimulation_pattern = [
    60,    # 1 minute no stim
    300,   # 5 minutes stim
    60,    # 1 minute no stim
    600,   # 10 minutes stim
    60,    # 1 minute no stim
    900,   # 15 minutes stim
    60,    # 1 minute no stim
    1200   # 20 minutes stim
    60     # 1 minute no stim
]

# Function to stimulate electrodes with transformed signals
def stimulate_electrodes(intan, trigger_gens, stim_params, transformed_signals, start_idx, duration):
    """
    Stimulate electrodes with transformed signals.
    
    Parameters:
    - intan: IntanSoftware instance to interact with the hardware.
    - trigger_gens: List of Trigger instances for sending trigger signals.
    - stim_params: List of StimParam objects configured for stimulation.
    - transformed_signals: Array of signals for direct stimulation.
    - start_idx: Starting index for the stimulation segment.
    - duration: Duration of the stimulation segment.
    """
    end_idx = start_idx + duration * intan.sampling_rate  # Calculate end index based on duration and sampling rate
    segment = transformed_signals[:, start_idx:end_idx]  # Extract the segment of the signals for the given duration

    for i, trigger_gen in enumerate(trigger_gens):
        for j in range(len(segment[i])):
            # Configure trigger for the current electrode
            trigger_key = stim_params[i].trigger_key
            trigger0 = np.zeros(16, dtype=np.uint8)  # Assuming 16 is correct for your setup
            trigger0[trigger_key] = 1

            # Send the trigger signal to the electrode
            intan.send_stimulus(trigger_gen, stim_params[i].index, segment[i, j])
            time.sleep(1 / intan.sampling_rate)  # Wait for the duration of one sample

            # Reset trigger for the next electrode
            trigger0[trigger_key] = 0
            trigger_gen.send(trigger0)

intan = IntanSoftware()
trigger_gens = [Trigger() for _ in range(num_stim_elecs)]  # One trigger generator per stimulator

stim_params = []  # Initialize an empty list for stimulation parameters

for i in range(num_stim_elecs):
    stim_param = StimParam()
    stim_param.index = i + 32  # Assuming electrodes 32-39 are used for stimulation
    stim_param.trigger_key = i  # Mapping each electrode to a trigger key, one per stimulator
    stim_params.append(stim_param)

try:
    if exp.start():
        intan.impedance()  # Measure impedance
        
        # Send stimulation parameters to the Intan hardware
        intan.send_stimparam(stim_params)

        # Start raw recording on channels selected
        raw_channels = np.array(range(num_read_elecs), dtype=np.int32) 
        intan.start_raw_recording(raw_channels)
        
        # Start recording stimulation data
        intan.start_stimulation_recording()

        start_idx = 0  # Initialize starting index for stimulation
        total_stim_data_length = 45 * 60 * intan.sampling_rate  # Total length of the stimulation data in samples

        for duration in stimulation_pattern:
            if duration > 0:
                if duration % 60 == 0:
                    # Stimulate for the specified duration
                    print(f"Stimulation for {duration // 60} minute(s)...")
                    stimulate_electrodes(intan, trigger_gens, stim_params, transformed_signals, start_idx, duration)
                    start_idx += duration * intan.sampling_rate  # Increment start index by the duration of samples
                    if start_idx >= total_stim_data_length:
                        print("End of stimulation data reached. Wrapping around to start.")
                        start_idx = 0  # Wrap around if the end of the data is reached
                else:
                    # No stim for the specified duration
                    print(f"No stimulation for {duration} seconds...")
                    time.sleep(duration)

finally:
    for trigger_gen in trigger_gens:
        trigger_gen.close()
    intan.stop_stimulation_recording()
    intan.close()
    exp.stop()